In [1]:
import sys
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm
%load_ext autoreload
import os
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.counts/ATEM/CZI5_NOVA_210816/')
! ls

CGE_ADARB2_ADAM33.total_reads.txt     L2-4IT_CUX2.minus.bam
Glia_Astro.minus.5pfilt.bam	      L2-4IT_CUX2.plus.bam
Glia_Astro.minus.5pfilt.bam.bai       L4-5IT_RORB_ARHGAP15.total_reads.txt
Glia_Astro.minus.5pfilt.metadata.txt  L6.minus.5pfilt.bam
Glia_Astro.minus.bam		      L6.minus.5pfilt.bam.bai
Glia_Astro.plus.5pfilt.bam	      L6.minus.bam
Glia_Astro.plus.5pfilt.bam.bai	      L6.plus.5pfilt.bam
Glia_Astro.plus.5pfilt.metadata.txt   L6.plus.5pfilt.bam.bai
Glia_Astro.plus.bam		      L6.plus.bam
Glia_Astro.total_reads.txt	      L6.total_reads.txt


In [3]:
import pyranges as pr
import pandas as pd
#BEWARY OF NAMING - PLUS AND MINUS!
#Read in required files for filtering
bed_pl = pd.read_csv('/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.plus.sort.bed', sep='\t', header=None)
bed_pl.columns =['Chromosome', 'Start', 'End', 'Strand', 'transcript_id', 'gene_id', 'family_id', 'class_id']
bed_mi = pd.read_csv('/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.minus.sort.bed',sep='\t', header=None)
bed_mi.columns =['Chromosome', 'Start', 'End', 'Strand', 'transcript_id', 'gene_id', 'family_id', 'class_id']

bam_pl = pr.read_bam('Glia_Astro.plus.5pfilt.bam', as_df=True) #CHANGE
bam_mi = pr.read_bam('Glia_Astro.minus.5pfilt.bam', as_df=True) #CHANGE
#Swap Start + End for minus strand
bam_mi['Start'], bam_mi['End'] = bam_mi['End'], bam_mi['Start']

#File checks
assert sum(bam_pl['Strand'] == '+') == len(bam_pl), 'Some non plus strands assigned to plus bam'
assert sum(bam_mi['Strand'] == '-') == len(bam_mi), 'Some non minus strands assigned to minus bam'
assert sum(bed_pl['Strand'] == '+') == len(bed_pl), 'Some non plus strands assigned to plus bed'
assert sum(bed_mi['Strand'] == '-') == len(bed_mi), 'Some non minus strands assigned to minus bed'


[W::hts_idx_load3] The index file is older than the data file: Glia_Astro.plus.5pfilt.bam.bai
[W::hts_idx_load3] The index file is older than the data file: Glia_Astro.plus.5pfilt.bam.bai


In [5]:
bam_mi

,Chromosome,Start,End,Strand,Flag
0,chr1,6394907,6394803,-,16
1,chr1,6394907,6394803,-,16
2,chr1,6394907,6394803,-,16
3,chr1,6394907,6394803,-,16
4,chr1,11055408,11055301,-,16
...,...,...,...,...,...
2921,chrX,151463535,151463412,-,16
2922,chrX,151463535,151463412,-,16
2923,chrY,22775090,22588560,-,272
2924,chrY,22775090,22588560,-,272


In [6]:
#Define + and - strand files
pl_pars = [bed_pl, bam_pl, 'Glia_Astro.plus.5pfilt.metadata.txt', 'plus']
mi_pars = [bed_mi, bam_mi, 'Glia_Astro.minus.5pfilt.metadata.txt', 'minus']
par_list = [pl_pars, mi_pars]


In [8]:
import numpy as np
curr_bam = pl_pars[1]
chr_unq = np.unique(curr_bam['Chromosome'].values)
chr_unq

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14',
       'chr14_KI270725v1_random', 'chr15', 'chr16',
       'chr16_KI270728v1_random', 'chr16_KI270853v1_alt',
       'chr16_KI270855v1_alt', 'chr17', 'chr17_GL000258v2_alt',
       'chr17_GL383563v3_alt', 'chr17_KI270857v1_alt',
       'chr17_KI270862v1_alt', 'chr17_KI270908v1_alt', 'chr18', 'chr19',
       'chr19_GL383573v1_alt', 'chr19_GL383574v1_alt',
       'chr1_KI270763v1_alt', 'chr2', 'chr20', 'chr20_KI270869v1_alt',
       'chr21', 'chr22', 'chr22_KI270876v1_alt', 'chr2_KI270769v1_alt',
       'chr3', 'chr3_KI270779v1_alt', 'chr3_KI270784v1_alt',
       'chr3_KI270935v1_alt', 'chr4', 'chr5', 'chr5_GL339449v2_alt',
       'chr5_KI270897v1_alt', 'chr6', 'chr6_GL000251v2_alt',
       'chr6_GL000252v2_alt', 'chr6_GL000253v2_alt',
       'chr6_GL000254v2_alt', 'chr6_GL000255v2_alt',
       'chr6_GL000256v2_alt', 'chr7', 'chr7_KI270806v1_alt', 'chr8',
       'chr8_KI270815v1_alt', 'chr8_KI270821v1_alt',
       'chr8_KI270822

In [68]:
%autoreload
import te_rna_f as te
pd.options.mode.chained_assignment = None  # default='warn'
ncount_df = pd.DataFrame() #empty count matrix
bam_ll = [[],[]] #empty list of lists to store curr_bam indices

# Filter out reads that do not overlap with 5' portion of insertion
for x,par in enumerate(par_list):
    curr_bed = par[0]
    curr_bam = par[1]
    curr_name = pd.read_csv(par[2], sep='\t', header=None)
    assert len(curr_bam) == len(curr_name), 'Bam and UMI files not the same length'
    curr_bam['UMI']=curr_name[0].values #CHANGE CHANGE CHANGE 

    #Loop through each chromosome
    chr_unq = np.unique(curr_bam['Chromosome'].values)
    chr = 'chr1'
    print('Aligning to chromosome ' + chr + ' for ' + par[3] + ' strand')
    #Slice bed/bam files by chromosome
    chr_bam = curr_bam[curr_bam['Chromosome'] == chr]
    chr_bed = curr_bed[curr_bed['Chromosome'] == chr]
    ncount_df, bam_ll[x] = te.five_prime_align(chr_bam, chr_bed, ncount_df, bam_ll[x])

# #Add in CPMs as a column
# total_reads = pd.read_csv('/cndd3/dburrows/DATA/te/rna/filt.prac/prac/total_reads.txt', sep=" ", header=None)[0].values[0]
# count_df['CPM'] = count_df['Count'].values / total_reads * 1000000

Aligning to chromosome chr1 for plus strand
Aligning to chromosome chr1 for minus strand


In [71]:
chr_bed


,Chromosome,Start,End,Strand,transcript_id,gene_id,family_id,class_id
0,chr1,23321,23421,-,MIR,MIR,SINE,258
1,chr1,62110,62210,-,AluSc,Alu,SINE,296
2,chr1,68196,68296,-,MIRb,MIR,SINE,262
3,chr1,87363,87463,-,AluJo,Alu,SINE,284
4,chr1,93981,94081,-,AluSc,Alu,SINE,302
...,...,...,...,...,...,...,...,...
46750,chr1,248920087,248920187,-,AluSx1,Alu,SINE,257
46751,chr1,248922307,248922407,-,AluY,Alu,SINE,309
46752,chr1,248926688,248926788,-,MIRb,MIR,SINE,211
46753,chr1,248935784,248935884,-,AluY,Alu,SINE,299


In [67]:
count_df

,Chromosome,Start,End,Strand,transcript_id,gene_id,family_id,class_id,Count
32077,chr1,164457935,164458035,+,AluSx,Alu,SINE,306,1.0
30407,chr1,155667167,155667267,+,AluJr,Alu,SINE,307,1.0
23015,chr1,94198145,94198245,+,AluJo,Alu,SINE,303,1.0
31178,chr1,159355372,159355472,+,AluY,Alu,SINE,304,1.0
47173,chr1,248835577,248835677,+,AluSz6,Alu,SINE,284,1.0
...,...,...,...,...,...,...,...,...,...
46311,chr1,246265377,246265477,-,AluJr4,Alu,SINE,276,1.0
39970,chr1,213073910,213074010,-,AluSx1,Alu,SINE,296,1.0
7421,chr1,25488299,25488399,-,AluJr4,Alu,SINE,285,1.0
29310,chr1,153696382,153696482,-,AluY,Alu,SINE,309,1.0


In [69]:
ncount_df

,Chromosome,Start,End,Strand,transcript_id,gene_id,family_id,class_id,Count
32077,chr1,164457935,164458035,+,AluSx,Alu,SINE,306,1.0
30407,chr1,155667167,155667267,+,AluJr,Alu,SINE,307,1.0
23015,chr1,94198145,94198245,+,AluJo,Alu,SINE,303,1.0
31178,chr1,159355372,159355472,+,AluY,Alu,SINE,304,1.0
47173,chr1,248835577,248835677,+,AluSz6,Alu,SINE,284,1.0
...,...,...,...,...,...,...,...,...,...
46311,chr1,246265377,246265477,-,AluJr4,Alu,SINE,276,1.0
39970,chr1,213073910,213074010,-,AluSx1,Alu,SINE,296,1.0
7421,chr1,25488299,25488399,-,AluJr4,Alu,SINE,285,1.0
29310,chr1,153696382,153696482,-,AluY,Alu,SINE,309,1.0


In [23]:
chr_bed

,Chromosome,Start,End,Strand,transcript_id,gene_id,family_id,class_id
0,chr1,23321,23421,-,MIR,MIR,SINE,258
1,chr1,62110,62210,-,AluSc,Alu,SINE,296
2,chr1,68196,68296,-,MIRb,MIR,SINE,262
3,chr1,87363,87463,-,AluJo,Alu,SINE,284
4,chr1,93981,94081,-,AluSc,Alu,SINE,302
...,...,...,...,...,...,...,...,...
46750,chr1,248920087,248920187,-,AluSx1,Alu,SINE,257
46751,chr1,248922307,248922407,-,AluY,Alu,SINE,309
46752,chr1,248926688,248926788,-,MIRb,MIR,SINE,211
46753,chr1,248935784,248935884,-,AluY,Alu,SINE,299


In [59]:
chr_bed

,Chromosome,Start,End,Strand,transcript_id,gene_id,family_id,class_id
194405,chr16_KI270855v1_alt,0,84,+,AluSc5,Alu,SINE,301
194406,chr16_KI270855v1_alt,728,828,+,AluY,Alu,SINE,309
194407,chr16_KI270855v1_alt,3031,3131,+,AluSc8,Alu,SINE,299
194408,chr16_KI270855v1_alt,3391,3491,+,MIR,MIR,SINE,204
194409,chr16_KI270855v1_alt,3645,3745,+,AluSx3,Alu,SINE,302
...,...,...,...,...,...,...,...,...
194493,chr16_KI270855v1_alt,225507,225607,+,AluSc,Alu,SINE,295
194494,chr16_KI270855v1_alt,227098,227198,+,AluJr,Alu,SINE,276
194495,chr16_KI270855v1_alt,228399,228499,+,AluSg,Alu,SINE,291
194496,chr16_KI270855v1_alt,228693,228793,+,AluY,Alu,SINE,307


In [61]:
np.ravel(np.asarray(all_bed_pos[:,0]))

array([array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
              17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
              34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
              51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
              68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]),
       array([728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740,
              741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753,
              754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766,
              767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779,
              780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792,
              793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805,
              806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818,
              819, 820, 821, 822, 8

In [44]:
np.asarray(all_bed_pos[:,0]).shape

(46755, 101)

In [51]:
sum(np.ravel(np.asarray(all_bed_pos[:,0,:]))  == np.ravel(np.asarray(all_bed_pos[:,0]))) == len(np.ravel(np.asarray(all_bed_pos[:,0,:])))

True

In [ ]:
{"in_path": "/cndd3/mcuoco/projects/czi_diversity_rnate/results/merge_bam", 
 
 "out_path": "/cndd3/dburrows/DATA/te/rna/CZI.counts/ATEM/pseudobulk",

In [2]:
import os
import glob
os.chdir('/cndd3/mcuoco/projects/czi_diversity_rnate/results/merge_bam/')
curr_l = glob.glob('**')
curr_l

['CZI10_NOVA_220113',
 'CZI20_NOVA_220113',
 'CZI13_NOVA_211107',
 'CZI2_NOVA_210611',
 'CZI22_NOVA_220113',
 'CZI14_NOVA_220113',
 'CZI8_NOVA_210905',
 'CZI5_NOVA_210816',
 'CZI19_NOVA_220113',
 'CZI16_NOVA_220113',
 'CZI4_NOVA_210816',
 'CZI9_NOVA_210905',
 'CZI1_NOVA_210502',
 'CZI15_NOVA_220113',
 'CZI17_NOVA_220113',
 'CZI18_NOVA_220113',
 'CZI21_NOVA_220113',
 'CZI11_NOVA_220113',
 'CZI23_NOVA_220113',
 'CZI3_NOVA_210611',
 'CZI12_NOVA_211107']

In [3]:
%autoreload
import os
import json
import glob
import numpy as np
os.chdir("/cndd3/dburrows/DATA/te/rna/CZI.counts/ATEM/pseudobulk/")

for curr in curr_l:
    os.mkdir(curr)

In [51]:
my_dict = {

    "in_path":"/cndd3/mcuoco/projects/czi_diversity_rnate/results/merge_bam",
    "out_path":"/cndd3/dburrows/DATA/te/rna/PE.counts/ATEM",
    "bed_plus_path":"/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.plus.sort.bed",
    "bed_minus_path":"/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.minus.sort.bed",

}



In [61]:
import os
import glob
os.chdir('/cndd3/mcuoco/projects/czi_diversity_rnate/results/merge_bam/')
GLU_dict = my_dict.copy()
GLU_dict['ID'] = glob.glob('**')
GLU_dict['bam'] = list(new_b)
GLU_dict
adm.make_config(GLU_dict, '/cndd3/dburrows/CODE/te_ageing/workflow.ATEM/config.CZI-pseudobulk.json')

Write successful


In [59]:
new_b = [bam_ll[i][:-4] for i in range(len(bam_ll))]

In [60]:
new_b

['CZI10_NOVA_220113/Glia_Astro',
 'CZI10_NOVA_220113/CGE_LAMP5',
 'CZI10_NOVA_220113/MGE_SST',
 'CZI10_NOVA_220113/L6',
 'CZI10_NOVA_220113/MGE_SST_CLMP',
 'CZI10_NOVA_220113/L6IT_THEMIS_CUX1',
 'CZI10_NOVA_220113/CGE_LAMP5_LHX6',
 'CZI10_NOVA_220113/L4-5IT_RORB_LRRK1',
 'CZI10_NOVA_220113/CGE_VIP',
 'CZI10_NOVA_220113/CGE_PAX6',
 'CZI10_NOVA_220113/L4-5IT_RORB_TSHZ2',
 'CZI10_NOVA_220113/L2-4IT_CUX2',
 'CZI10_NOVA_220113/L56NP_TLE4_TSHZ2',
 'CZI10_NOVA_220113/L4-5IT_RORB_ARHGAP15',
 'CZI10_NOVA_220113/Glia_Oligo',
 'CZI10_NOVA_220113/MGE_PVALB',
 'CZI10_NOVA_220113/CGE_ADARB2_ADAM33',
 'CZI10_NOVA_220113/L6IT_THEMIS_LINC00343',
 'CZI10_NOVA_220113/L3-5IT_RORB_PLCH1',
 'CZI20_NOVA_220113/MGE_SST_CLMP',
 'CZI20_NOVA_220113/L3-5IT_RORB_PLCH1',
 'CZI20_NOVA_220113/L4-5IT_RORB_TSHZ2',
 'CZI20_NOVA_220113/Glia_Oligo',
 'CZI20_NOVA_220113/CGE_ADARB2_ADAM33',
 'CZI20_NOVA_220113/MGE_PVALB',
 'CZI20_NOVA_220113/L2-4IT_CUX2',
 'CZI20_NOVA_220113/L4-5IT_RORB_ARHGAP15',
 'CZI20_NOVA_220113/MGE_SS

In [34]:
os.chdir('/cndd3/mcuoco/projects/czi_diversity_rnate/results/merge_bam/')
curr_l = glob.glob('**')
bam_ll = []
for curr in curr_l:
    bam_l = glob.glob(curr+'/*.bam')
    print(len(bam_l))
    bam_ll = np.append(bam_ll, bam_l)

19
19
19
19
18
19
19
19
19
19
19
19
19
19
18
19
18
19
19
19
19


In [12]:
import os
import glob
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam')
l = glob.glob('*Samp**')
for n in l:
    os.mkdir('/cndd3/dburrows/DATA/te/rna/PE.counts/ATEM/pseudobulk' + n)

In [2]:
import pandas as pd
bed_pl = pd.read_csv("/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.plus.sort.bed", sep='\t', header=None)



In [4]:
bed_pl

,0,1,2,3,4,5,6,7
0,chr1,24038,24138,+,MIR,MIR,SINE,211
1,chr1,26741,26841,+,AluSp,Alu,SINE,263
2,chr1,31386,31486,+,AluJo,Alu,SINE,297
3,chr1,32791,32891,+,MIR,MIR,SINE,214
4,chr1,39574,39674,+,AluSx,Alu,SINE,300
...,...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE,308
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE,305
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE,278
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE,308


In [7]:
bed_pl.columns =['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'family_id', 'class_id', 'length']
bed_pl

,Chromosome,Start,End,Strand,gene_id,family_id,class_id,length
0,chr1,24038,24138,+,MIR,MIR,SINE,211
1,chr1,26741,26841,+,AluSp,Alu,SINE,263
2,chr1,31386,31486,+,AluJo,Alu,SINE,297
3,chr1,32791,32891,+,MIR,MIR,SINE,214
4,chr1,39574,39674,+,AluSx,Alu,SINE,300
...,...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE,308
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE,305
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE,278
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE,308


In [8]:
#drop last columns
bed_pl = bed_pl.drop(columns=['length'])
bed_pl

,Chromosome,Start,End,Strand,gene_id,family_id,class_id
0,chr1,24038,24138,+,MIR,MIR,SINE
1,chr1,26741,26841,+,AluSp,Alu,SINE
2,chr1,31386,31486,+,AluJo,Alu,SINE
3,chr1,32791,32891,+,MIR,MIR,SINE
4,chr1,39574,39674,+,AluSx,Alu,SINE
...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE


In [1]:
#Check that all files are present
import glob
import os

full_l = os.listdir('/cndd2/Public_Datasets/Dracheva_PsychEncode_development/raw_May2022/') 
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam/')
check_l = glob.glob('*Samp*')

import numpy as np
not_int = np.setxor1d(check_l, full_l)
assert len(full_l) == len(check_l)
assert len(not_int) == 0

In [ ]:
STAR --genomeDir /cndd2/jchien/iGenome/STAR_gencode_v37 --readFilesIn  R1-merge.fastq.gz R1-merge.fastq.gz --outSAMunmapped None --outFilterType BySJout --outSAMattributes All --outFilterMultimapNmax 100 --outFilterMismatchNmax 999 --outFilterMismatchNoverReadLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --sjdbScore 1 --readFilesCommand zcat --runThreadN 16--genomeLoad LoadAndKeep --limitBAMsortRAM 10000000000 --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --winAnchorMultimapNmax 200 --outMultimapperOrder Random --outSAMmultNmax -1

In [76]:
# Check which files went to completion
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam/')
check_l = glob.glob('*Samp*')
for c in check_l:
    os.chdir(c)
    if check_log('Log.out', 'ALL DONE') == False:
        print(c + ' incomplete Log file')
    
    if 'Aligned.sortedByCoord.out.bam.bai' not in os.listdir():
        print(c + ' missing index file')

    os.chdir('..')